# Horcrux Joystick 입력 학습 진행

## 필요 패키지 import

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

# 조이스틱 환경 삽입
from horcrux_terrain_v1.envs import PlaneJoyWorld
from horcrux_terrain_v1.envs import PlaneWorld

# Ray 패키지 삽입
import ray
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.sac import SACConfig

from ray.tune.registry import register_env

## Ray 실행

In [2]:
# import socket
# import psutil

# conn_ip = ""
# interfaces = psutil.net_if_addrs()
# for interface_name, addresses in interfaces.items():
#     if "openvpn" in interface_name.lower() and "tap" in interface_name.lower():
#         snicaddrs = interfaces[str(interface_name)]
#         for addrfamily in snicaddrs:
#             if addrfamily.family == socket.AF_INET:
#                 conn_ip = addrfamily.address

# print(f"Connection IP: {conn_ip}")
# # 해당 init을 통해서 VPN을 통한 외부 접속 가능함.
# ray.init(dashboard_host=conn_ip, dashboard_port=8265)

ray.init()

2025-03-13 15:21:11,550	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.11
Ray version:,2.36.1
Dashboard:,http://127.0.0.1:8265


(RolloutWorker pid=39896) Stack (most recent call first):
(RolloutWorker pid=39896)   File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\ctypes\__init__.py", line 376 in __init__
(RolloutWorker pid=39896)   File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\mujoco\__init__.py", line 183 in _load_all_bundled_plugins
(RolloutWorker pid=39896)   File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\mujoco\__init__.py", line 190 in <module>
(RolloutWorker pid=39896)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed
(RolloutWorker pid=39896)   File "<frozen importlib._bootstrap_external>", line 940 in exec_module
(RolloutWorker pid=39896)   File "<frozen importlib._bootstrap>", line 690 in _load_unlocked
(RolloutWorker pid=39896)   File "<frozen importlib._bootstrap>", line 1147 in _find_and_load_unlocked
(RolloutWorker pid=39896)   File "<frozen importlib._bootstrap>", line 1176 in _find_and_load
(RolloutWorker pid=39896)   File "c:\Users\doore\anac

## Gym 환경 등록하기

In [3]:
env_config = {
    "forward_reward_weight": 6.5,
    "side_cost_weight": 2.0,
    "unhealthy_max_steps": 100,
    "healthy_reward": 0.5,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.01,
    "rotation_orientation_cost_weight": 1.2,
    "termination_reward": 0,
    "gait_params": (30, 30, 60, 60, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
}

# JoyWorld
register_env("joy-v1", lambda config: PlaneJoyWorld( forward_reward_weight=env_config["forward_reward_weight"], 
                                                     side_cost_weight=env_config["side_cost_weight"], 
                                                     unhealthy_max_steps=env_config["unhealthy_max_steps"],
                                                     healthy_reward=env_config["healthy_reward"], 
                                                     healthy_roll_range=env_config["healthy_roll_range"],
                                                     terminating_roll_range=env_config["terminating_roll_range"],
                                                     rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                     rotation_orientation_cost_weight=env_config["rotation_orientation_cost_weight"],
                                                     termination_reward=env_config["termination_reward"],
                                                     gait_params=env_config["gait_params"],
                                                     use_friction_chg=env_config["use_friction_chg"],
                                                     joy_input_random=env_config["joy_input_random"],
                                                   )
            )

# Plane
register_env("plane-v1", lambda config: PlaneWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_reward=env_config["healthy_reward"],
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 terminating_roll_range=env_config["terminating_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 rotation_orientation_cost_weight=env_config["rotation_orientation_cost_weight"],
                                                 termination_reward=env_config["termination_reward"],
                                                 use_friction_chg=env_config["use_friction_chg"],
                                                 gait_params=env_config["gait_params"]))

## 학습된 알고리즘 불러오기

In [ ]:
# algo = Algorithm.from_checkpoint("./Paper_agents/good/Linear/SAC_layer_512_5_32_Linear_restart_final")

# trained_weights = algo.get_weights()
# algo.cleanup()

## 학습 알고리즘 설정하기

In [4]:
config = SACConfig()

# 구형 API 구조 사용
config.api_stack(
    enable_rl_module_and_learner=False,
    enable_env_runner_and_connector_v2=False,
)

config.environment("joy-v1")
config.framework("torch")

# 병렬 CPU 사용 설정
total_workers = 16
# config.resources(num_gpus=1,num_cpus_per_worker=1, num_gpus_per_worker= 1/(total_workers+1))
config.env_runners(num_env_runners = total_workers, num_cpus_per_env_runner = 1, num_gpus_per_env_runner = 0.3/(total_workers))
config.training(
    gamma=0.95,
    replay_buffer_config={
    "_enable_replay_buffer_api": True,
    "capacity": int(4e7),
    # If True prioritized replay buffer will be used.
    "prioritized_replay": False,
    "prioritized_replay_alpha": 0.6,
    "prioritized_replay_beta": 0.4,
    "prioritized_replay_eps": 1e-6,
    # Whether to compute priorities already on the remote worker side.
    "worker_side_prioritization": False,
    },

    q_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define custom Q-model(s).
            "custom_model_config": {},
    },
    policy_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define a custom policy model.
            "custom_model_config": {},
    },

    train_batch_size_per_learner = 4096,
    num_steps_sampled_before_learning_starts = 4096,
)

algo = config.build()


c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\rllib\algorithms\algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be rem

RaySystemError: System error: [WinError 1114] DLL 초기화 루틴을 실행할 수 없습니다
traceback: Traceback (most recent call last):
  File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\_private\serialization.py", line 423, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\_private\serialization.py", line 280, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\_private\serialization.py", line 235, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\doore\anaconda3\envs\gdtor2\Lib\site-packages\ray\_private\serialization.py", line 225, in

## 학습시작

In [ ]:
from pprint import pprint

n_iter = 63000
save_iter = 0
save_name = "SAC_layer_512_5_32_linear_friction_joy_31"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}", end=", ")
    # result.pop("config")
    # pprint(result)

    if i%3000 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

algo.save(save_name+str("_final"))

# 학습 알고리즘 평가하기

In [ ]:
algo = Algorithm.from_checkpoint("./SAC_layer_512_5_32_linear_friction_joy2_2") # 학습된 정책 로드

## 평가용 Env 생성 및 실행

In [ ]:
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
from ray.rllib.algorithms.algorithm import Algorithm
import time

eval_config = env_config
eval_config["use_friction_chg"] = False
eval_config["joy_input_random"] = False
eval_config["joy_input"] = (1, 0, 0)


In [ ]:
env = gym.make("horcrux_terrain_v1/plane-v2", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
               render_camera_name = 'ceiling', 
               use_gait = True,               
               **eval_config,
               ) 

for j in range(3):
   episode_return = 0
   yaw_total = 0
   terminated = truncated = False

   obs, info = env.reset()

   for i in range(1000):

      action = algo.compute_single_action(observation= obs)
      
      obs, reward, terminated, truncated, info = env.step(action)
      if i == 0:
          print(f"Joy input = {info['joy_input']}")
      if terminated:
         print("terminated")

      episode_return += reward
      yaw_total += info['step_ypr'][0] / 0.1

   print(f"Reached episode return of {episode_return}.")
   print(yaw_total)

env.close()

### Pytorch 모델 바로 불러오기

In [ ]:
import torch
from ray.rllib.policy.policy import Policy
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2

_algo = Algorithm.from_checkpoint("./Paper_agents/good/Linear/SAC_layer_512_5_32_Linear_restart_final")
# _policy = Policy.from_checkpoint("./SAC_layer_512_5_32_linear_friction_joy_31_0")
# _policy = Policy.from_checkpoint("./Paper_agents/good/Linear/SAC_layer_512_5_32_Linear_restart_final")


In [ ]:
randobs = np.random.random(94)

# _algo.compute_single_action(randobs)
# _algo.get_policy('default_policy').dist_class.mro()
_algo.get_policy('default_policy').model.action_dist



In [ ]:
randobs = np.random.random(94)

# obs = torch.tensor(randobs, dtype=torch.float32)
_policy['default_policy'].dist_class
# _policy['default_policy'].compute_single_action(torch.tensor(randobs, dtype=torch.float32), explore=False)